In [ ]:
import tkinter as tk
from tkinter import ttk, filedialog, messagebox, scrolledtext
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure
from datetime import datetime, timedelta
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.arima.model import ARIMA
import seaborn as sns
import os
from fpdf import FPDF
import warnings

# Suprimir advertencias de statsmodels
warnings.filterwarnings("ignore")

class VentasApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Sistema Inteligente de Análisis de Ventas - LocalMarket AI")
        self.root.geometry("1200x800")
        self.root.minsize(1000, 700)
        
        # Variables de estado
        self.data = None
        self.current_analysis = None
        self.report_data = {
            'analisis': {},
            'graficos': [],
            'recomendaciones': []
        }
        
        # Configurar estilo
        self.setup_styles()
        
        # Crear interfaz
        self.create_welcome_frame()
        
    def setup_styles(self):
        """Configura los estilos visuales de la aplicación"""
        style = ttk.Style()
        style.configure('TFrame', background='#f0f0f0')
        style.configure('TLabel', background='#f0f0f0', font=('Arial', 10))
        style.configure('TButton', font=('Arial', 10), padding=5)
        style.configure('Header.TLabel', font=('Arial', 14, 'bold'))
        style.configure('Title.TLabel', font=('Arial', 18, 'bold'))
        style.configure('Report.TLabel', font=('Arial', 11), wraplength=600)
        
    def clear_frame(self):
        """Limpia el frame actual"""
        for widget in self.root.winfo_children():
            widget.destroy()
    
    def create_welcome_frame(self):
        """Crea la pantalla de bienvenida"""
        self.clear_frame()
        
        # Frame principal
        main_frame = ttk.Frame(self.root, padding="20")
        main_frame.pack(fill=tk.BOTH, expand=True)
        
        # Logo/Title
        ttk.Label(main_frame, text="LocalMarket AI", style='Title.TLabel').pack(pady=20)
        ttk.Label(main_frame, text="Sistema Inteligente de Análisis y Proyección de Ventas", 
                 style='Header.TLabel').pack(pady=10)
        
        # Descripción
        desc_text = (
            "Esta herramienta le permite analizar sus datos de ventas históricos, generar proyecciones "
            "a futuro y obtener recomendaciones estratégicas basadas en inteligencia artificial.\n\n"
            "Cargue sus datos para comenzar el análisis."
        )
        ttk.Label(main_frame, text=desc_text, style='Report.TLabel', justify=tk.CENTER).pack(pady=20)
        
        # Botones de acción
        btn_frame = ttk.Frame(main_frame)
        btn_frame.pack(pady=30)
        
        ttk.Button(btn_frame, text="Cargar Datos", command=self.create_data_loader_frame).pack(side=tk.LEFT, padx=10)
        ttk.Button(btn_frame, text="Usar Datos de Ejemplo", command=self.load_example_data).pack(side=tk.LEFT, padx=10)
        ttk.Button(btn_frame, text="Salir", command=self.root.quit).pack(side=tk.LEFT, padx=10)
    
    def create_data_loader_frame(self):
        """Crea la interfaz para cargar datos"""
        self.clear_frame()
        
        # Frame principal
        main_frame = ttk.Frame(self.root, padding="20")
        main_frame.pack(fill=tk.BOTH, expand=True)
        
        # Título
        ttk.Label(main_frame, text="Carga de Datos", style='Title.TLabel').pack(pady=10)
        
        # Controles de carga
        load_frame = ttk.Frame(main_frame)
        load_frame.pack(pady=20, fill=tk.X)
        
        ttk.Label(load_frame, text="Seleccione archivo de datos:").pack(side=tk.LEFT, padx=5)
        self.file_entry = ttk.Entry(load_frame, width=50)
        self.file_entry.pack(side=tk.LEFT, padx=5)
        ttk.Button(load_frame, text="Examinar", command=self.browse_file).pack(side=tk.LEFT, padx=5)
        
        # Opciones avanzadas
        adv_frame = ttk.LabelFrame(main_frame, text="Opciones Avanzadas", padding=10)
        adv_frame.pack(pady=10, fill=tk.X)
        
        ttk.Label(adv_frame, text="Formato del archivo:").grid(row=0, column=0, sticky=tk.W)
        self.file_format = ttk.Combobox(adv_frame, values=["CSV", "Excel", "JSON"])
        self.file_format.grid(row=0, column=1, sticky=tk.W, padx=5)
        self.file_format.set("CSV")
        
        ttk.Label(adv_frame, text="Columna de fecha:").grid(row=1, column=0, sticky=tk.W)
        self.date_col_entry = ttk.Entry(adv_frame, width=20)
        self.date_col_entry.grid(row=1, column=1, sticky=tk.W, padx=5)
        
        # Botones de acción
        btn_frame = ttk.Frame(main_frame)
        btn_frame.pack(pady=20)
        
        ttk.Button(btn_frame, text="Cargar Datos", command=self.load_data).pack(side=tk.LEFT, padx=10)
        ttk.Button(btn_frame, text="Regresar", command=self.create_welcome_frame).pack(side=tk.LEFT, padx=10)
        
        # Vista previa
        self.preview_text = scrolledtext.ScrolledText(main_frame, height=10, wrap=tk.NONE)
        self.preview_text.pack(fill=tk.BOTH, expand=True, pady=10)
        
        # Configurar barras de desplazamiento
        x_scroll = ttk.Scrollbar(main_frame, orient=tk.HORIZONTAL, command=self.preview_text.xview)
        x_scroll.pack(fill=tk.X)
        self.preview_text.configure(xscrollcommand=x_scroll.set)
    
    def browse_file(self):
        """Abre el diálogo para seleccionar archivo"""
        filetypes = [
            ('Archivos CSV', '*.csv'),
            ('Archivos Excel', '*.xlsx'),
            ('Archivos JSON', '*.json'),
            ('Todos los archivos', '*.*')
        ]
        
        filename = filedialog.askopenfilename(filetypes=filetypes)
        if filename:
            self.file_entry.delete(0, tk.END)
            self.file_entry.insert(0, filename)
            
            # Mostrar vista previa
            self.show_file_preview(filename)
    
    def show_file_preview(self, filename):
        """Muestra una vista previa del archivo seleccionado"""
        try:
            # Determinar tipo de archivo
            if filename.endswith('.csv'):
                preview = pd.read_csv(filename, nrows=10)
            elif filename.endswith('.xlsx'):
                preview = pd.read_excel(filename, nrows=10)
            elif filename.endswith('.json'):
                preview = pd.read_json(filename, lines=True, nrows=10)
            else:
                self.preview_text.delete(1.0, tk.END)
                self.preview_text.insert(tk.END, "Formato de archivo no soportado para vista previa")
                return
            
            self.preview_text.delete(1.0, tk.END)
            self.preview_text.insert(tk.END, preview.to_string())
            
        except Exception as e:
            messagebox.showerror("Error", f"No se pudo leer el archivo:\n{str(e)}")
    
    def load_data(self):
        """Carga los datos del archivo seleccionado"""
        filename = self.file_entry.get()
        if not filename:
            messagebox.showwarning("Advertencia", "Seleccione un archivo primero")
            return
        
        try:
            if filename.endswith('.csv'):
                self.data = pd.read_csv(filename)
            elif filename.endswith('.xlsx'):
                self.data = pd.read_excel(filename)
            elif filename.endswith('.json'):
                self.data = pd.read_json(filename)
            else:
                messagebox.showerror("Error", "Formato de archivo no soportado")
                return
            
            # Procesar fechas si se especificó una columna
            date_col = self.date_col_entry.get()
            if date_col and date_col in self.data.columns:
                try:
                    self.data[date_col] = pd.to_datetime(self.data[date_col])
                except:
                    messagebox.showwarning("Advertencia", f"No se pudo convertir la columna '{date_col}' a fecha")
            
            messagebox.showinfo("Éxito", f"Datos cargados correctamente\nRegistros: {len(self.data)}")
            self.create_analysis_frame()
            
        except Exception as e:
            messagebox.showerror("Error", f"No se pudo cargar el archivo:\n{str(e)}")
    
    def load_example_data(self):
        """Carga datos de ejemplo"""
        try:
            # Crear datos de ejemplo similares a los que proporcionaste
            dates = pd.date_range(end=datetime.today(), periods=365*3, freq='D')
            products = ['Cuadernos', 'Leche', 'Clavos', 'Pintura', 'Juguetes']
            regions = ['Norte', 'Sur', 'Este', 'Oeste', 'Centro']
            
            data = {
                'Fecha': np.random.choice(dates, 1000),
                'Producto': np.random.choice(products, 1000),
                'Cantidad': np.random.randint(1, 100, 1000),
                'Precio': np.round(np.random.uniform(10, 500, 1000), 2),
                'Región': np.random.choice(regions, 1000),
                'Tipo_Zona': np.random.choice(['Urbano', 'Rural', 'Periurbano'], 1000),
                'Temporada': np.random.choice(['Alta', 'Baja', 'Festiva'], 1000)
            }
            
            self.data = pd.DataFrame(data)
            messagebox.showinfo("Éxito", "Datos de ejemplo cargados correctamente")
            self.create_analysis_frame()
            
        except Exception as e:
            messagebox.showerror("Error", f"No se pudieron generar datos de ejemplo:\n{str(e)}")
    
    def create_analysis_frame(self):
        """Crea la interfaz de análisis de datos"""
        self.clear_frame()
        
        # Frame principal con paneles
        main_frame = ttk.Frame(self.root)
        main_frame.pack(fill=tk.BOTH, expand=True)
        
        # Panel izquierdo (controles)
        control_frame = ttk.Frame(main_frame, width=300, padding="10")
        control_frame.pack(side=tk.LEFT, fill=tk.Y)
        control_frame.pack_propagate(False)
        
        # Panel derecho (resultados)
        result_frame = ttk.Frame(main_frame, padding="10")
        result_frame.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True)
        
        # Controles de análisis
        ttk.Label(control_frame, text="Opciones de Análisis", style='Header.TLabel').pack(pady=10)
        
        # Tipo de análisis
        ttk.Label(control_frame, text="Tipo de Análisis:").pack(anchor=tk.W)
        self.analysis_type = ttk.Combobox(control_frame, values=[
            "Análisis Descriptivo",
            "Tendencias Temporales",
            "Análisis por Producto",
            "Análisis por Región",
            "Análisis de Temporadas",
            "Proyección de Ventas"
        ])
        self.analysis_type.pack(fill=tk.X, pady=5)
        
        # Configuración específica del análisis
        self.analysis_config_frame = ttk.Frame(control_frame)
        self.analysis_config_frame.pack(fill=tk.X, pady=10)
        self.setup_analysis_config()
        
        # Botones de acción
        ttk.Button(control_frame, text="Ejecutar Análisis", command=self.execute_analysis).pack(fill=tk.X, pady=5)
        ttk.Button(control_frame, text="Generar Reporte", command=self.generate_report).pack(fill=tk.X, pady=5)
        ttk.Button(control_frame, text="Volver", command=self.create_welcome_frame).pack(fill=tk.X, pady=5)
        
        # Resultados
        self.result_notebook = ttk.Notebook(result_frame)
        self.result_notebook.pack(fill=tk.BOTH, expand=True)
        
        # Pestañas de resultados
        self.tab_text = tk.Text(self.result_notebook)
        self.tab_graph = ttk.Frame(self.result_notebook)
        self.tab_table = ttk.Frame(self.result_notebook)
        
        self.result_notebook.add(self.tab_text, text="Resumen")
        self.result_notebook.add(self.tab_graph, text="Gráficos")
        self.result_notebook.add(self.tab_table, text="Datos")
        
        # Configurar área de texto
        self.text_output = scrolledtext.ScrolledText(self.tab_text, wrap=tk.WORD)
        self.text_output.pack(fill=tk.BOTH, expand=True)
        
        # Configurar área de gráficos
        self.graph_frame = ttk.Frame(self.tab_graph)
        self.graph_frame.pack(fill=tk.BOTH, expand=True)
        
        # Configurar tabla de datos
        self.table_frame = ttk.Frame(self.tab_table)
        self.table_frame.pack(fill=tk.BOTH, expand=True)
        
        # Mostrar resumen inicial
        self.show_data_summary()
    
    def setup_analysis_config(self):
        """Configura los controles específicos para el tipo de análisis seleccionado"""
        # Limpiar frame de configuración
        for widget in self.analysis_config_frame.winfo_children():
            widget.destroy()
        
        analysis_type = self.analysis_type.get()
        
        if not analysis_type:
            return
        
        if analysis_type == "Proyección de Ventas":
            ttk.Label(self.analysis_config_frame, text="Período de Proyección:").pack(anchor=tk.W)
            self.projection_period = ttk.Combobox(self.analysis_config_frame, values=[
                "1 Mes", "3 Meses", "6 Meses", "1 Año", "2 Años", "3 Años"
            ])
            self.projection_period.pack(fill=tk.X, pady=5)
            self.projection_period.set("6 Meses")
            
            ttk.Label(self.analysis_config_frame, text="Método de Proyección:").pack(anchor=tk.W)
            self.projection_method = ttk.Combobox(self.analysis_config_frame, values=[
                "Regresión Lineal", "Promedio Móvil", "ARIMA", "Tendencia Estacional"
            ])
            self.projection_method.pack(fill=tk.X, pady=5)
            self.projection_method.set("Regresión Lineal")
            
            ttk.Label(self.analysis_config_frame, text="Agrupar por:").pack(anchor=tk.W)
            self.group_by = ttk.Combobox(self.analysis_config_frame, values=["Producto", "Región", "Tipo_Zona", "Temporada"])
            self.group_by.pack(fill=tk.X, pady=5)
        
        elif analysis_type == "Análisis por Producto":
            ttk.Label(self.analysis_config_frame, text="Seleccionar Producto:").pack(anchor=tk.W)
            products = self.data['Producto'].unique() if 'Producto' in self.data.columns else []
            self.product_select = ttk.Combobox(self.analysis_config_frame, values=products)
            self.product_select.pack(fill=tk.X, pady=5)
            
            ttk.Label(self.analysis_config_frame, text="Mostrar:").pack(anchor=tk.W)
            self.product_metric = ttk.Combobox(self.analysis_config_frame, values=[
                "Ventas Totales", "Ventas Promedio", "Tendencia Temporal", "Comparación por Región"
            ])
            self.product_metric.pack(fill=tk.X, pady=5)
        
        # Configuraciones similares para otros tipos de análisis...
    
    def show_data_summary(self):
        """Muestra un resumen de los datos cargados"""
        if self.data is None:
            return
        
        summary = []
        summary.append("=== RESUMEN DE DATOS ===")
        summary.append(f"Total de registros: {len(self.data)}")
        summary.append(f"Columnas disponibles: {', '.join(self.data.columns)}")
        summary.append("\nEstadísticas descriptivas:")
        
        # Agregar estadísticas descriptivas para columnas numéricas
        numeric_cols = self.data.select_dtypes(include=['int64', 'float64']).columns
        if not numeric_cols.empty:
            summary.append(self.data[numeric_cols].describe().to_string())
        else:
            summary.append("No se encontraron columnas numéricas para análisis.")
        
        # Mostrar en el área de texto
        self.text_output.delete(1.0, tk.END)
        self.text_output.insert(tk.END, "\n".join(summary))
        
        # Generar gráfico inicial
        self.show_initial_graph()
    
    def show_initial_graph(self):
        """Muestra un gráfico inicial de los datos"""
        # Limpiar frame de gráficos
        for widget in self.graph_frame.winfo_children():
            widget.destroy()
        
        fig = Figure(figsize=(8, 4), dpi=100)
        ax = fig.add_subplot(111)
        
        # Intentar graficar datos temporales si existe una columna de fecha
        date_cols = [col for col in self.data.columns if 'fecha' in col.lower() or 'date' in col.lower()]
        
        if date_cols and 'Cantidad' in self.data.columns:
            try:
                # Usar la primera columna de fecha encontrada
                temp_data = self.data.copy()
                temp_data['Fecha'] = pd.to_datetime(temp_data[date_cols[0]])
                temp_data = temp_data.sort_values('Fecha')
                
                # Agrupar por fecha si hay muchas fechas únicas
                if len(temp_data['Fecha'].unique()) > 30:
                    temp_data = temp_data.groupby(pd.Grouper(key='Fecha', freq='M'))['Cantidad'].sum().reset_index()
                    ax.plot(temp_data['Fecha'], temp_data['Cantidad'], label='Ventas Mensuales')
                else:
                    ax.plot(temp_data['Fecha'], temp_data['Cantidad'], label='Ventas Diarias')
                
                ax.set_title('Tendencia de Ventas')
                ax.set_xlabel('Fecha')
                ax.set_ylabel('Cantidad Vendida')
                ax.legend()
                ax.grid(True)
                
            except:
                # Si falla, mostrar gráfico de barras de productos
                if 'Producto' in self.data.columns:
                    product_counts = self.data['Producto'].value_counts().head(10)
                    product_counts.plot(kind='bar', ax=ax)
                    ax.set_title('Productos Más Vendidos')
                    ax.set_xlabel('Producto')
                    ax.set_ylabel('Cantidad')
        elif 'Producto' in self.data.columns:
            product_counts = self.data['Producto'].value_counts().head(10)
            product_counts.plot(kind='bar', ax=ax)
            ax.set_title('Productos Más Vendidos')
            ax.set_xlabel('Producto')
            ax.set_ylabel('Cantidad')
        else:
            ax.text(0.5, 0.5, 'No hay datos suficientes para generar gráfico', 
                   ha='center', va='center', transform=ax.transAxes)
        
        # Integrar gráfico en la interfaz
        canvas = FigureCanvasTkAgg(fig, master=self.graph_frame)
        canvas.draw()
        canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)
    
    def execute_analysis(self):
        """Ejecuta el análisis seleccionado"""
        analysis_type = self.analysis_type.get()
        
        if not analysis_type:
            messagebox.showwarning("Advertencia", "Seleccione un tipo de análisis")
            return
        
        try:
            if analysis_type == "Proyección de Ventas":
                self.run_sales_projection()
            elif analysis_type == "Análisis Descriptivo":
                self.run_descriptive_analysis()
            elif analysis_type == "Análisis por Producto":
                self.run_product_analysis()
            # Agregar otros tipos de análisis...
            
        except Exception as e:
            messagebox.showerror("Error", f"No se pudo completar el análisis:\n{str(e)}")
    
    def run_sales_projection(self):
        """Ejecuta proyección de ventas"""
        period = self.projection_period.get()
        method = self.projection_method.get()
        group_by = self.group_by.get() if self.group_by.get() else None
        
        # Determinar períodos a proyectar
        if period == "1 Mes":
            periods = 1
            freq = 'M'
        elif period == "3 Meses":
            periods = 3
            freq = 'M'
        elif period == "6 Meses":
            periods = 6
            freq = 'M'
        elif period == "1 Año":
            periods = 12
            freq = 'M'
        elif period == "2 Años":
            periods = 24
            freq = 'M'
        else:  # 3 Años
            periods = 36
            freq = 'M'
        
        # Preparar datos
        if 'Fecha' not in self.data.columns:
            # Si no hay columna de fecha, usar índice como tiempo
            temp_data = self.data.copy()
            temp_data['Fecha'] = pd.date_range(end=datetime.today(), periods=len(temp_data), freq='D')
        else:
            temp_data = self.data.copy()
            temp_data['Fecha'] = pd.to_datetime(temp_data['Fecha'])
        
        # Agrupar datos si es necesario
        if group_by:
            grouped_data = temp_data.groupby([pd.Grouper(key='Fecha', freq='M'), group_by])['Cantidad'].sum().unstack()
        else:
            grouped_data = temp_data.groupby(pd.Grouper(key='Fecha', freq='M'))['Cantidad'].sum()
        
        # Realizar proyección según método seleccionado
        if method == "Regresión Lineal":
            projection = self.linear_regression_projection(grouped_data, periods)
        elif method == "Promedio Móvil":
            projection = self.moving_average_projection(grouped_data, periods)
        elif method == "ARIMA":
            projection = self.arima_projection(grouped_data, periods)
        else:  # Tendencia Estacional
            projection = self.seasonal_trend_projection(grouped_data, periods)
        
        # Mostrar resultados
        self.show_projection_results(projection, group_by)
        
        # Guardar resultados para el reporte
        self.current_analysis = {
            'type': 'Proyección de Ventas',
            'period': period,
            'method': method,
            'group_by': group_by,
            'results': projection
        }
    
    def linear_regression_projection(self, data, periods):
        """Proyección usando regresión lineal"""
        if isinstance(data, pd.DataFrame):
            # Proyección para múltiples series (agrupadas)
            projections = {}
            for col in data.columns:
                series = data[col].dropna()
                if len(series) < 2:
                    continue
                
                X = np.arange(len(series)).reshape(-1, 1)
                y = series.values
                
                model = LinearRegression()
                model.fit(X, y)
                
                # Predecir futuros períodos
                future_X = np.arange(len(series), len(series) + periods).reshape(-1, 1)
                future_y = model.predict(future_X)
                
                projections[col] = future_y
            
            return projections
        else:
            # Proyección para serie única
            series = data.dropna()
            if len(series) < 2:
                return None
                
            X = np.arange(len(series)).reshape(-1, 1)
            y = series.values
            
            model = LinearRegression()
            model.fit(X, y)
            
            # Predecir futuros períodos
            future_X = np.arange(len(series), len(series) + periods).reshape(-1, 1)
            future_y = model.predict(future_X)
            
            return future_y
    
    def moving_average_projection(self, data, periods, window=3):
        """Proyección usando promedio móvil"""
        if isinstance(data, pd.DataFrame):
            projections = {}
            for col in data.columns:
                series = data[col].dropna()
                if len(series) < window:
                    continue
                
                # Calcular promedio móvil
                ma = series.rolling(window=window).mean()
                last_ma = ma.iloc[-1]
                
                # Proyectar usando el último promedio móvil
                future_y = [last_ma] * periods
                projections[col] = future_y
            
            return projections
        else:
            series = data.dropna()
            if len(series) < window:
                return None
                
            # Calcular promedio móvil
            ma = series.rolling(window=window).mean()
            last_ma = ma.iloc[-1]
            
            # Proyectar usando el último promedio móvil
            future_y = [last_ma] * periods
            
            return future_y
    
    def arima_projection(self, data, periods, order=(1,1,1)):
        """Proyección usando modelo ARIMA"""
        if isinstance(data, pd.DataFrame):
            projections = {}
            for col in data.columns:
                series = data[col].dropna()
                if len(series) < 5:  # Mínimo para ARIMA
                    continue
                
                try:
                    model = ARIMA(series, order=order)
                    model_fit = model.fit()
                    future_y = model_fit.forecast(steps=periods)
                    projections[col] = future_y
                except:
                    continue
            
            return projections
        else:
            series = data.dropna()
            if len(series) < 5:
                return None
                
            try:
                model = ARIMA(series, order=order)
                model_fit = model.fit()
                future_y = model_fit.forecast(steps=periods)
                return future_y
            except:
                return None
    
    def seasonal_trend_projection(self, data, periods):
        """Proyección considerando tendencia estacional (simplificado)"""
        if isinstance(data, pd.DataFrame):
            projections = {}
            for col in data.columns:
                series = data[col].dropna()
                if len(series) < 12:  # Necesitamos al menos 1 año de datos
                    continue
                
                # Calcular promedio por mes (estacionalidad)
                monthly_avg = series.groupby(series.index.month).mean()
                
                # Calcular tendencia lineal
                X = np.arange(len(series)).reshape(-1, 1)
                y = series.values
                model = LinearRegression()
                model.fit(X, y)
                trend = model.predict(X)
                
                # Desestacionalizar
                deseasonalized = series.copy()
                for month in range(1, 13):
                    mask = series.index.month == month
                    deseasonalized[mask] = series[mask] - monthly_avg[month]
                
                # Proyectar tendencia
                future_X = np.arange(len(series), len(series) + periods).reshape(-1, 1)
                future_trend = model.predict(future_X)
                
                # Añadir estacionalidad
                future_months = pd.date_range(start=series.index[-1], periods=periods+1, freq='M')[1:].month
                future_season = [monthly_avg[m] for m in future_months]
                future_y = future_trend + future_season
                
                projections[col] = future_y
            
            return projections
        else:
            series = data.dropna()
            if len(series) < 12:
                return None
                
            # Calcular promedio por mes (estacionalidad)
            monthly_avg = series.groupby(series.index.month).mean()
            
            # Calcular tendencia lineal
            X = np.arange(len(series)).reshape(-1, 1)
            y = series.values
            model = LinearRegression()
            model.fit(X, y)
            trend = model.predict(X)
            
            # Desestacionalizar
            deseasonalized = series.copy()
            for month in range(1, 13):
                mask = series.index.month == month
                deseasonalized[mask] = series[mask] - monthly_avg[month]
            
            # Proyectar tendencia
            future_X = np.arange(len(series), len(series) + periods).reshape(-1, 1)
            future_trend = model.predict(future_X)
            
            # Añadir estacionalidad
            future_months = pd.date_range(start=series.index[-1], periods=periods+1, freq='M')[1:].month
            future_season = [monthly_avg[m] for m in future_months]
            future_y = future_trend + future_season
            
            return future_y
    
    def show_projection_results(self, projection, group_by=None):
        """Muestra los resultados de la proyección"""
        # Limpiar áreas de resultados
        self.text_output.delete(1.0, tk.END)
        for widget in self.graph_frame.winfo_children():
            widget.destroy()
        
        # Generar resumen textual
        summary = ["=== RESULTADOS DE PROYECCIÓN ==="]
        
        if projection is None:
            summary.append("No se pudo generar la proyección con los datos disponibles.")
        elif isinstance(projection, dict):
            summary.append(f"Proyección agrupada por: {group_by}")
            for group, values in projection.items():
                summary.append(f"\n{group}:")
                for i, val in enumerate(values, 1):
                    summary.append(f"  Mes {i}: {val:.1f} unidades estimadas")
        else:
            summary.append("Proyección de ventas totales:")
            for i, val in enumerate(projection, 1):
                summary.append(f"Mes {i}: {val:.1f} unidades estimadas")
        
        self.text_output.insert(tk.END, "\n".join(summary))
        
        # Generar gráfico
        fig = Figure(figsize=(8, 5), dpi=100)
        ax = fig.add_subplot(111)
        
        if isinstance(projection, dict):
            # Gráfico para proyección agrupada
            df = pd.DataFrame(projection)
            df.plot(kind='line', ax=ax, marker='o')
            ax.set_title(f'Proyección de Ventas por {group_by}')
        elif projection is not None:
            # Gráfico para proyección simple
            ax.plot(range(len(projection)), projection, marker='o', label='Proyección')
            ax.set_title('Proyección de Ventas Totales')
        
        ax.set_xlabel('Períodos Futuros')
        ax.set_ylabel('Unidades Estimadas')
        ax.legend()
        ax.grid(True)
        
        # Integrar gráfico en la interfaz
        canvas = FigureCanvasTkAgg(fig, master=self.graph_frame)
        canvas.draw()
        canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)
        
        # Generar recomendaciones basadas en la proyección
        self.generate_recommendations(projection, group_by)
    
    def generate_recommendations(self, projection, group_by=None):
        """Genera recomendaciones basadas en los resultados de la proyección"""
        recommendations = []
        
        if projection is None:
            recommendations.append("No se pueden generar recomendaciones sin una proyección válida.")
        else:
            recommendations.append("=== RECOMENDACIONES ESTRATÉGICAS ===")
            
            if isinstance(projection, dict):
                # Análisis para datos agrupados
                total_projection = {k: sum(v) for k, v in projection.items()}
                best_group = max(total_projection.items(), key=lambda x: x[1])[0]
                worst_group = min(total_projection.items(), key=lambda x: x[1])[0]
                
                recommendations.append(f"\nEl {group_by.lower()} con mayor proyección es: {best_group}")
                recommendations.append(f"Considere aumentar recursos/inventario para este {group_by.lower()}.")
                
                recommendations.append(f"\nEl {group_by.lower()} con menor proyección es: {worst_group}")
                recommendations.append(f"Evalúe estrategias de marketing o promociones para este {group_by.lower()}.")
                
                # Variabilidad entre grupos
                variability = np.std(list(total_projection.values())) / np.mean(list(total_projection.values()))
                if variability > 0.3:
                    recommendations.append("\nAlta variabilidad entre grupos. Considere estrategias diferenciadas.")
                else:
                    recommendations.append("\nBaja variabilidad entre grupos. Estrategias uniformes pueden ser adecuadas.")
            else:
                # Análisis para proyección simple
                trend = projection[-1] - projection[0]
                
                if trend > 0:
                    recommendations.append("\nTendencia positiva detectada. Considere expandir capacidad.")
                elif trend < 0:
                    recommendations.append("\nTendencia negativa detectada. Evalúe causas y considere promociones.")
                else:
                    recommendations.append("\nTendencia estable. Mantenga estrategias actuales.")
                
                # Variabilidad en la proyección
                variability = np.std(projection) / np.mean(projection)
                if variability > 0.2:
                    recommendations.append("Alta variabilidad en la proyección. Mercado potencialmente volátil.")
        
        # Mostrar recomendaciones en una nueva pestaña si no existe
        if "Recomendaciones" not in self.result_notebook.tabs():
            tab_rec = ttk.Frame(self.result_notebook)
            self.result_notebook.add(tab_rec, text="Recomendaciones")
            
            text_rec = scrolledtext.ScrolledText(tab_rec, wrap=tk.WORD)
            text_rec.pack(fill=tk.BOTH, expand=True)
            text_rec.insert(tk.END, "\n".join(recommendations))
        else:
            # Actualizar pestaña existente
            for child in self.result_notebook.winfo_children():
                if self.result_notebook.tab(child, "text") == "Recomendaciones":
                    text_rec = child.winfo_children()[0]
                    text_rec.delete(1.0, tk.END)
                    text_rec.insert(tk.END, "\n".join(recommendations))
                    break
        
        # Guardar para el reporte
        self.report_data['recomendaciones'] = recommendations
    
    def run_descriptive_analysis(self):
        """Ejecuta análisis descriptivo de los datos"""
        # Limpiar áreas de resultados
        self.text_output.delete(1.0, tk.END)
        for widget in self.graph_frame.winfo_children():
            widget.destroy()
        
        # Generar resumen estadístico
        summary = ["=== ANÁLISIS DESCRIPTIVO ==="]
        
        # Estadísticas para columnas numéricas
        numeric_cols = self.data.select_dtypes(include=['int64', 'float64']).columns
        if not numeric_cols.empty:
            summary.append("\nEstadísticas Numéricas:")
            summary.append(self.data[numeric_cols].describe().to_string())
        else:
            summary.append("\nNo se encontraron columnas numéricas para análisis.")
        
        # Estadísticas para columnas categóricas
        cat_cols = self.data.select_dtypes(include=['object', 'category']).columns
        if not cat_cols.empty:
            summary.append("\n\nEstadísticas Categóricas:")
            for col in cat_cols:
                summary.append(f"\n{col}:")
                summary.append(f"  Valores únicos: {self.data[col].nunique()}")
                top_values = self.data[col].value_counts().head(5)
                summary.append("  Valores más frecuentes:")
                summary.append(top_values.to_string())
        else:
            summary.append("\nNo se encontraron columnas categóricas para análisis.")
        
        self.text_output.insert(tk.END, "\n".join(summary))
        
        # Generar gráficos
        fig = Figure(figsize=(10, 8), dpi=100)
        
        # Gráfico de distribución para columnas numéricas
        if not numeric_cols.empty:
            ax1 = fig.add_subplot(211)
            self.data[numeric_cols].boxplot(ax=ax1)
            ax1.set_title('Distribución de Variables Numéricas')
            ax1.tick_params(axis='x', rotation=45)
            
            # Gráfico de barras para categóricas
            if not cat_cols.empty:
                ax2 = fig.add_subplot(212)
                top_cat = cat_cols[0] if len(cat_cols) > 0 else None
                if top_cat:
                    value_counts = self.data[top_cat].value_counts().head(10)
                    value_counts.plot(kind='bar', ax=ax2)
                    ax2.set_title(f'Distribución de {top_cat}')
                    ax2.tick_params(axis='x', rotation=45)
        else:
            # Si no hay numéricas, mostrar solo categóricas
            ax = fig.add_subplot(111)
            top_cat = cat_cols[0] if len(cat_cols) > 0 else None
            if top_cat:
                value_counts = self.data[top_cat].value_counts().head(10)
                value_counts.plot(kind='bar', ax=ax)
                ax.set_title(f'Distribución de {top_cat}')
                ax.tick_params(axis='x', rotation=45)
            else:
                ax.text(0.5, 0.5, 'No hay datos adecuados para gráficos', 
                       ha='center', va='center', transform=ax.transAxes)
        
        # Integrar gráfico en la interfaz
        canvas = FigureCanvasTkAgg(fig, master=self.graph_frame)
        canvas.draw()
        canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)
        
        # Guardar para el reporte
        self.current_analysis = {
            'type': 'Análisis Descriptivo',
            'results': summary
        }
    
    def run_product_analysis(self):
        """Ejecuta análisis por producto"""
        product = self.product_select.get()
        metric = self.product_metric.get()
        
        if not product or not metric:
            messagebox.showwarning("Advertencia", "Seleccione un producto y métrica")
            return
        
        # Limpiar áreas de resultados
        self.text_output.delete(1.0, tk.END)
        for widget in self.graph_frame.winfo_children():
            widget.destroy()
        
        # Filtrar datos por producto
        product_data = self.data[self.data['Producto'] == product]
        
        # Generar resumen
        summary = [f"=== ANÁLISIS PARA PRODUCTO: {product} ==="]
        
        if metric == "Ventas Totales":
            total_sales = product_data['Cantidad'].sum()
            total_revenue = (product_data['Cantidad'] * product_data['Precio']).sum()
            
            summary.append(f"\nVentas Totales: {total_sales} unidades")
            summary.append(f"Ingresos Totales: ${total_revenue:,.2f}")
            summary.append(f"Precio Promedio: ${product_data['Precio'].mean():,.2f}")
            
            # Gráfico de ventas por tiempo si hay fecha
            fig = Figure(figsize=(8, 4), dpi=100)
            ax = fig.add_subplot(111)
            
            if 'Fecha' in product_data.columns:
                try:
                    temp_data = product_data.copy()
                    temp_data['Fecha'] = pd.to_datetime(temp_data['Fecha'])
                    temp_data = temp_data.sort_values('Fecha')
                    
                    # Agrupar por mes si hay muchos datos
                    if len(temp_data) > 30:
                        grouped = temp_data.groupby(pd.Grouper(key='Fecha', freq='M'))['Cantidad'].sum()
                        grouped.plot(ax=ax, marker='o')
                        ax.set_title(f'Ventas Mensuales de {product}')
                    else:
                        temp_data.plot(x='Fecha', y='Cantidad', ax=ax, marker='o')
                        ax.set_title(f'Ventas Diarias de {product}')
                    
                    ax.set_ylabel('Cantidad Vendida')
                    ax.grid(True)
                except:
                    product_data['Cantidad'].plot(kind='hist', ax=ax)
                    ax.set_title(f'Distribución de Ventas de {product}')
            else:
                product_data['Cantidad'].plot(kind='hist', ax=ax)
                ax.set_title(f'Distribución de Ventas de {product}')
            
        elif metric == "Ventas Promedio":
            avg_sales = product_data['Cantidad'].mean()
            std_sales = product_data['Cantidad'].std()
            
            summary.append(f"\nVentas Promedio: {avg_sales:.1f} unidades")
            summary.append(f"Desviación Estándar: {std_sales:.1f}")
            summary.append(f"Rango de Ventas: {product_data['Cantidad'].min()} - {product_data['Cantidad'].max()}")
            
            # Gráfico de distribución
            fig = Figure(figsize=(8, 4), dpi=100)
            ax = fig.add_subplot(111)
            
            sns.histplot(product_data['Cantidad'], kde=True, ax=ax)
            ax.set_title(f'Distribución de Ventas de {product}')
            ax.set_xlabel('Cantidad Vendida')
            ax.grid(True)
        
        elif metric == "Tendencia Temporal":
            if 'Fecha' not in product_data.columns:
                messagebox.showwarning("Advertencia", "No hay datos de fecha para análisis temporal")
                return
            
            try:
                temp_data = product_data.copy()
                temp_data['Fecha'] = pd.to_datetime(temp_data['Fecha'])
                temp_data = temp_data.sort_values('Fecha')
                
                # Agrupar por mes
                monthly_sales = temp_data.groupby(pd.Grouper(key='Fecha', freq='M'))['Cantidad'].sum()
                
                summary.append("\nTendencia Temporal:")
                summary.append(monthly_sales.describe().to_string())
                
                # Gráfico de serie temporal
                fig = Figure(figsize=(10, 4), dpi=100)
                ax = fig.add_subplot(111)
                
                monthly_sales.plot(ax=ax, marker='o')
                ax.set_title(f'Tendencia de Ventas Mensuales para {product}')
                ax.set_ylabel('Cantidad Vendida')
                ax.grid(True)
                
                # Añadir línea de tendencia
                X = np.arange(len(monthly_sales)).reshape(-1, 1)
                y = monthly_sales.values
                model = LinearRegression()
                model.fit(X, y)
                trend = model.predict(X)
                ax.plot(monthly_sales.index, trend, 'r--', label='Tendencia')
                ax.legend()
                
            except Exception as e:
                messagebox.showerror("Error", f"No se pudo analizar tendencia temporal:\n{str(e)}")
                return
        
        elif metric == "Comparación por Región":
            if 'Región' not in product_data.columns:
                messagebox.showwarning("Advertencia", "No hay datos de región para comparación")
                return
            
            region_sales = product_data.groupby('Región')['Cantidad'].sum().sort_values(ascending=False)
            
            summary.append("\nVentas por Región:")
            summary.append(region_sales.to_string())
            
            # Gráfico de barras
            fig = Figure(figsize=(8, 4), dpi=100)
            ax = fig.add_subplot(111)
            
            region_sales.plot(kind='bar', ax=ax)
            ax.set_title(f'Ventas de {product} por Región')
            ax.set_ylabel('Cantidad Vendida')
            ax.tick_params(axis='x', rotation=45)
            ax.grid(True)
        
        self.text_output.insert(tk.END, "\n".join(summary))
        
        # Integrar gráfico si existe
        if 'fig' in locals():
            canvas = FigureCanvasTkAgg(fig, master=self.graph_frame)
            canvas.draw()
            canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)
        
        # Guardar para el reporte
        self.current_analysis = {
            'type': 'Análisis por Producto',
            'product': product,
            'metric': metric,
            'results': summary
        }
    
    def generate_report(self):
        """Genera un reporte con los resultados del análisis"""
        if not self.current_analysis:
            messagebox.showwarning("Advertencia", "No hay análisis reciente para generar reporte")
            return
        
        # Crear diálogo para seleccionar formato
        report_dialog = tk.Toplevel(self.root)
        report_dialog.title("Generar Reporte")
        report_dialog.geometry("400x300")
        
        ttk.Label(report_dialog, text="Seleccione formato de reporte:", style='Header.TLabel').pack(pady=10)
        
        # Opciones de formato
        format_var = tk.StringVar(value="PDF")
        
        ttk.Radiobutton(report_dialog, text="PDF", variable=format_var, value="PDF").pack(anchor=tk.W, padx=50)
        ttk.Radiobutton(report_dialog, text="Excel", variable=format_var, value="Excel").pack(anchor=tk.W, padx=50)
        ttk.Radiobutton(report_dialog, text="CSV", variable=format_var, value="CSV").pack(anchor=tk.W, padx=50)
        ttk.Radiobutton(report_dialog, text="Imagen (PNG)", variable=format_var, value="PNG").pack(anchor=tk.W, padx=50)
        
        ttk.Label(report_dialog, text="Nombre del archivo:").pack(pady=5)
        name_entry = ttk.Entry(report_dialog, width=30)
        name_entry.pack()
        name_entry.insert(0, f"reporte_{datetime.now().strftime('%Y%m%d')}")
        
        # Botones de acción
        btn_frame = ttk.Frame(report_dialog)
        btn_frame.pack(pady=20)
        
        ttk.Button(btn_frame, text="Generar", 
                  command=lambda: self.save_report(format_var.get(), name_entry.get(), report_dialog)).pack(side=tk.LEFT, padx=10)
        ttk.Button(btn_frame, text="Cancelar", command=report_dialog.destroy).pack(side=tk.LEFT, padx=10)
    
    def save_report(self, format_type, filename, dialog):
        """Guarda el reporte en el formato seleccionado"""
        try:
            # Crear directorio de reportes si no existe
            if not os.path.exists("reportes"):
                os.makedirs("reportes", exist_ok=True)
                
            
            full_path = os.path.join("reportes", filename)
            
            if format_type == "PDF":
                self.generate_pdf_report(full_path + ".pdf")
            elif format_type == "Excel":
                self.generate_excel_report(full_path + ".xlsx")
            elif format_type == "CSV":
                self.generate_csv_report(full_path + ".csv")
            elif format_type == "PNG":
                self.save_graph_as_image(full_path + ".png")
            
            messagebox.showinfo("Éxito", f"Reporte generado correctamente en: reportes/{filename}.{format_type.lower()}")
            dialog.destroy()
            
        except Exception as e:
            messagebox.showerror("Error", f"No se pudo generar el reporte:\n{str(e)}")
    
    def generate_pdf_report(self, filepath):
        """Genera un reporte en formato PDF"""
        pdf = FPDF(orientation="L")  # Usar orientación horizontal (landscape)
        pdf.add_page()
        pdf.set_font("Arial", size=10)  # Reducir tamaño de fuente
        
        # Título
        pdf.set_font("Arial", 'B', 16)
        pdf.cell(200, 10, txt="Reporte de Análisis de Ventas", ln=1, align='C')
        pdf.set_font("Arial", size=12)
        pdf.cell(200, 10, txt=f"Fecha: {datetime.now().strftime('%Y-%m-%d %H:%M')}", ln=1)
        pdf.ln(10)
        
        # Información del análisis
        pdf.set_font("Arial", 'B', 14)
        pdf.cell(200, 10, txt=f"Tipo de Análisis: {self.current_analysis['type']}", ln=1)
        pdf.set_font("Arial", size=12)
        
        # Agregar detalles específicos del análisis
        if self.current_analysis['type'] == "Proyección de Ventas":
            pdf.cell(200, 10, txt=f"Método: {self.current_analysis['method']}", ln=1)
            pdf.cell(200, 10, txt=f"Período: {self.current_analysis['period']}", ln=1)
            if self.current_analysis['group_by']:
                pdf.cell(200, 10, txt=f"Agrupado por: {self.current_analysis['group_by']}", ln=1)
        
        elif self.current_analysis['type'] == "Análisis por Producto":
            pdf.cell(200, 10, txt=f"Producto: {self.current_analysis['product']}", ln=1)
            pdf.cell(200, 10, txt=f"Métrica: {self.current_analysis['metric']}", ln=1)
        
        pdf.ln(10)
        
        # Resultados
        pdf.set_font("Arial", 'B', 14)
        pdf.cell(200, 10, txt="Resultados:", ln=1)
        pdf.set_font("Arial", size=12)
        
        if isinstance(self.current_analysis['results'], list):
            for line in self.current_analysis['results']:
                pdf.multi_cell(0, 10, txt=line)
        else:
            pdf.multi_cell(0, 10, txt=str(self.current_analysis['results']))
        
        pdf.ln(10)
        
        # Recomendaciones
        if 'recomendaciones' in self.report_data and self.report_data['recomendaciones']:
            pdf.set_font("Arial", 'B', 14)
            pdf.cell(200, 10, txt="Recomendaciones:", ln=1)
            pdf.set_font("Arial", size=12)
            
            for rec in self.report_data['recomendaciones']:
                pdf.multi_cell(0, 10, txt=rec)
        
        # Guardar PDF
        pdf.output(filepath)
    
    def generate_excel_report(self, filepath):
        """Genera un reporte en formato Excel"""
        writer = pd.ExcelWriter(filepath, engine='xlsxwriter')
        
        # Crear hoja de resumen
        summary_data = []
        
        if self.current_analysis['type'] == "Proyección de Ventas":
            summary_data.append(["Tipo de Análisis", self.current_analysis['type']])
            summary_data.append(["Método", self.current_analysis['method']])
            summary_data.append(["Período", self.current_analysis['period']])
            if self.current_analysis['group_by']:
                summary_data.append(["Agrupado por", self.current_analysis['group_by']])
            
            # Agregar resultados de proyección
            if isinstance(self.current_analysis['results'], dict):
                for group, values in self.current_analysis['results'].items():
                    for i, val in enumerate(values, 1):
                        summary_data.append([f"{group} - Mes {i}", val])
            else:
                for i, val in enumerate(self.current_analysis['results'], 1):
                    summary_data.append([f"Mes {i}", val])
        
        elif self.current_analysis['type'] == "Análisis Descriptivo":
            summary_data.append(["Tipo de Análisis", self.current_analysis['type']])
            for line in self.current_analysis['results']:
                if line.strip() and not line.startswith("==="):
                    summary_data.append([line.split(":")[0].strip(), line.split(":")[1].strip() if ":" in line else ""])
        
        # Crear DataFrame y guardar
        pd.DataFrame(summary_data).to_excel(writer, sheet_name="Resumen", index=False, header=False)
        
        # Guardar datos originales en otra hoja
        self.data.to_excel(writer, sheet_name="Datos Originales", index=False)
        
        writer.close()
    
    def generate_csv_report(self, filepath):
        """Genera un reporte en formato CSV"""
        if isinstance(self.current_analysis['results'], dict):
            pd.DataFrame(self.current_analysis['results']).to_csv(filepath)
        else:
            with open(filepath, 'w') as f:
                if isinstance(self.current_analysis['results'], list):
                    f.write("\n".join(self.current_analysis['results']))
                else:
                    f.write(str(self.current_analysis['results']))
    
    def save_graph_as_image(self, filepath):
        """Guarda el gráfico actual como imagen"""
        # Obtener el gráfico actual de la pestaña de gráficos
        for widget in self.graph_frame.winfo_children():
            if isinstance(widget, FigureCanvasTkAgg):
                widget.figure.savefig(filepath)
                break

if __name__ == "__main__":
    root = tk.Tk()
    app = VentasApp(root)
    root.mainloop()